In [ ]:
from xmlrpc.client import DateTime
from telethon.sync import TelegramClient
import configparser
import json

from telethon.sync import TelegramClient
from datetime import datetime
from telethon.tl.functions.messages import GetDialogsRequest
from telethon.tl.types import InputPeerEmpty
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import PeerChannel

import csv


api_id = your api id
api_hash = 'your api hash'
phone = 'your phone number'

client = TelegramClient(phone, api_id, api_hash)

client.start()

async def parseMessages(channel):
    offset_msg = 0
    limit_msg = 100

    all_messages = []
    total_messages = 0
    total_count_limit = 0

    class DateTimeEncoder(json.JSONEncoder):

        def default(self, o):
            if isinstance(o, datetime):
                return o.isoformat()
            if isinstance(o, bytes):
                return list(o)
            return json.JSONEncoder.default(self, o)

    while True:
        history = await client(GetHistoryRequest(
            peer=channel,
            offset_id=offset_msg,
            offset_date=None, add_offset=0,
            limit=limit_msg, max_id=0, min_id=0,
            hash=0))
        if not history.messages:
            break
        messages = history.messages
        for message in messages:
            all_messages.append(message.to_dict())
        offset_msg = messages[len(messages) - 1].id
        total_messages = len(all_messages)
        if total_count_limit != 0 and total_messages >= total_count_limit:
            break

    with open('channel_messages1.json', 'w', encoding='utf8') as outfile:
        json.dump(all_messages, outfile, ensure_ascii=False, cls=DateTimeEncoder)

config = configparser.ConfigParser()
config.read("config.ini")


async def main():
    url = "https://t.me/Sputnik_results"
    channel = await client.get_entity(url)
    await parseMessages(channel)


with client:
    client.loop.run_until_complete(main())